# Setup de l'environnement


In [10]:
# Base comparative méthode Reinert: https://cran.r-project.org/web/packages/rainette/vignettes/algorithmes.html
# Etude de la présence/absence inter documentaire
# -------------------------------------------------------------------------------------------------------------

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words_en = stopwords.words()
stop_words_fr = stopwords.words("french")
stop_words = stop_words_en + stop_words_fr
# Dépendance de punkt pour le modèle de tokenisation
nltk.download("punkt_tab")
import spacy

nlp = spacy.load("fr_core_news_md")
import pandas as pd
import numpy as np
import string
import re
from unidecode import unidecode

from collections import Counter
from spacy.symbols import NOUN, VERB, ADJ, ADV

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\web\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# Import du fichier de départ

In [12]:
# Import du fichier
dataset_path = "files/ph-th-entretiens.xlsx"
df = pd.read_excel(dataset_path)
df

,question,reponse
0,Quelle définition donneriez-vous de l'opinion ...,"La définition de l’opinion publique, selon moi..."
1,Que pensez-vous de l'influence des outils digi...,"Je dirais que l’influence, enfin je dirais tou..."
2,Est-ce que vous arrivez à récolter facilement ...,"Il y a un élément qui nous le rappelle, qui no..."
3,Pourriez-vous vous passez d'un site internet d...,"Moi je pense, c’est ce que j’ai dit tout à l’h..."
4,"Est-ce que vous participez, ou avez participé,...","Oui bien sûr, j’ai déjà participé à des webcam..."
...,...,...
191,Est-ce que la base militante est difficile à m...,Ça dépend des élections. Ça dépend du contexte...
192,Que pensez-vous de l'utilisation des plateform...,"En fait j’ai toujours trouvé ça, mais c’est un..."
193,Est-ce que vous pensez que les boucles sont un...,"Oui, je pense que les boucles sont une bonne s..."
194,Comment se déroule la planification des action...,"C’est très, très divers. C’est un peu dans le ..."


# Processing

In [13]:
# Compilation des regex en constantes
DIGITS_PATTERN = re.compile(r'\d')
WHITESPACE_PATTERN = re.compile(r'\s+')
PUNCTUATION_TRANS = str.maketrans(string.punctuation, ' ' * len(string.punctuation))

def uci(value):
    """
    Nettoie une chaîne de caractères en supprimant les chiffres, accents,
    ponctuations et stop words.
    
    Args:
        value (str): Chaîne à nettoyer
        
    Returns:
        str: Chaîne nettoyée
    """
    if not isinstance(value, str):
        return ""
        
    try:
        # Suppression des chiffres et conversion en minuscules
        value = DIGITS_PATTERN.sub('', value.lower())
        
        # Suppression des accents
        value = unidecode(value)
        
        # Suppression de la ponctuation et des espaces multiples
        value = WHITESPACE_PATTERN.sub(' ', value.translate(PUNCTUATION_TRANS).strip())
        
        # Filtrage des mots
        return ' '.join(
            word for word in value.split()
            if len(word) >= 3 and word not in stop_words
        )

    except Exception as e:
        logger.error(f"Erreur lors du nettoyage UCI: {str(e)}")
        return ""

In [14]:
# Création des UCI : unité de contexte initial
# Tagging des métadonnées : Le début de l'énnoncé est marqué par *NomVariable"  ex : *Partie_l *chapitre_l_l
# Création des UCI avec gestion complète des métadonnées
import logging

# Configuration du logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Compilation du pattern pour les métadonnées
METADATA_PATTERN = re.compile(r'(\*[A-Za-z_]+\d*)')

# Création des UCI avec métadonnées
def process_uci_with_metadata(text):
    """
    Traite le texte en préservant les métadonnées et en appliquant le nettoyage UCI
    
    Args:
        text (str): Texte à traiter
    Returns:
        str: Texte nettoyé avec métadonnées préservées
    """
    if not isinstance(text, str):
        return ""
    
    # Extraction des métadonnées
    metadata = METADATA_PATTERN.findall(text)
    
    # Nettoyage du texte principal
    cleaned_text = uci(text)
    
    # Réinsertion des métadonnées au début
    if metadata:
        return ' '.join(metadata) + ' ' + cleaned_text
    return cleaned_text

# Application du traitement sur le DataFrame
logger.info("Début du traitement des UCI...")
df['uci'] = df['reponse'].astype(str).map(process_uci_with_metadata)

# Création des colonnes séparées pour analyse
df['metadata'] = df['uci'].str.extract(r'^((?:\*[A-Za-z_]+\d*\s*)+)')
df['clean_text'] = df['uci'].str.replace(r'^\s*(?:\*[A-Za-z_]+\d*\s*)+', '')

# Analyse des métadonnées
metadata_stats = df['uci'].str.extractall(r'(\*[A-Za-z_]+\d*)').value_counts()

# Affichage des statistiques
logger.info("Statistiques des métadonnées trouvées:")
print(metadata_stats)

# Affichage des premiers résultats
print("\nExemple des 5 premières UCI avec leurs métadonnées:")
for idx, row in df.head().iterrows():
    metadata = METADATA_PATTERN.findall(row['uci'])
    if metadata:
        print(f"Row {idx} - Metadata: {metadata}")
    print(f"UCI: {row['uci']}\n")

logger.info("Traitement des UCI terminé.")

INFO:__main__:Début du traitement des UCI...
INFO:__main__:Statistiques des métadonnées trouvées:
INFO:__main__:Traitement des UCI terminé.


Series([], Name: count, dtype: int64)

Exemple des 5 premières UCI avec leurs métadonnées:
UCI: definition opinion publique selon instant pense public vis vis sujet occurrence opinion publique politique pense comment positionne gens quand dis gens grand public peut etre associations chefs entreprise donc definition donne opinion public comment opinion public mesure taux satisfaction regard question posee

UCI: dirais influence enfin dirais tout simplement reseaux sociaux digital presents communication politique autant pris part importante communication politique parce comme certains disent faut vivre temps neanmoins faut savoir mesurer veux dire campagne politique evidemment reseaux sociaux evidemment numerique rien mieux aussi contact gens mesurer tout cela dire outil numerique vient complementarite autres outils evidemment rappele pandemie accelere impose modifie relation pouvait avoir puisqu campagne coller affiches contact distribuer tracts marches rencontre individus quel pendant 

In [15]:
# Définition des UCE
# Création des UCE à partir des UCI
import re
import logging
import nltk
from nltk.tokenize import sent_tokenize

# Configuration du logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Compilation du pattern pour les métadonnées
METADATA_PATTERN = re.compile(r'(\*[A-Za-z_]+\d*)')

def uce(text: str) -> str:
    """
    Crée les Unités de Contexte Élémentaire (UCE) à partir du texte
    Une UCE correspond généralement à une phrase ou un segment significatif
    
    Args:
        text (str): Texte à traiter
    Returns:
        str: Texte segmenté en UCE
    """
    try:
        if not isinstance(text, str) or not text.strip():
            return ""
        
        # Téléchargement du modèle de tokenization si nécessaire
        try:
            nltk.data.find('tokenizers/punkt')
        except LookupError:
            nltk.download('punkt')
        
        # Segmentation en phrases
        sentences = sent_tokenize(text, language='french')
        
        # Nettoyage et filtrage des phrases
        cleaned_sentences = []
        for sentence in sentences:
            # Nettoyage basique
            cleaned = sentence.strip()
            
            # Filtres de validation
            if (len(cleaned.split()) >= 3 and  # Au moins 3 mots
                any(c.isalpha() for c in cleaned)):  # Au moins une lettre
                cleaned_sentences.append(cleaned)
        
        # Jointure des phrases valides
        return ' '.join(cleaned_sentences)
        
    except Exception as e:
        logger.error(f"Erreur lors de la création des UCE: {str(e)}")
        return text

def process_uce_from_uci(text: str) -> str:
    """
    Traite le texte UCI pour créer l'UCE en préservant les métadonnées
    
    Args:
        text (str): Texte UCI à traiter
    Returns:
        str: Texte UCE avec métadonnées préservées
    """
    if not isinstance(text, str):
        return ""
    
    # Extraction des métadonnées existantes
    metadata = METADATA_PATTERN.findall(text)
    
    # Application de l'UCE sur le texte sans les métadonnées
    cleaned_text = text
    if metadata:
        # Retire temporairement les métadonnées pour le traitement
        cleaned_text = METADATA_PATTERN.sub('', text).strip()
    
    # Application du traitement UCE
    processed_text = uce(cleaned_text)
    
    # Réinsertion des métadonnées
    if metadata:
        return ' '.join(metadata) + ' ' + processed_text
    return processed_text

# Application sur le DataFrame
try:
    logger.info("Début du traitement des UCE...")
    
    # Création des UCE
    df['uce'] = df['uci'].astype(str).map(process_uce_from_uci)
    
    # Analyse des résultats
    logger.info("Analyse des résultats UCE...")
    print("\nExemple des 5 premiers résultats UCI → UCE:")
    for idx, row in df.head().iterrows():
        print(f"\nRow {idx}:")
        print(f"UCI: {row['uci']}")
        print(f"UCE: {row['uce']}")
    
    # Statistiques de comparaison UCI vs UCE
    df['uci_length'] = df['uci'].str.split().str.len()
    df['uce_length'] = df['uce'].str.split().str.len()
    print("\nStatistiques de réduction:")
    print(f"Moyenne mots UCI: {df['uci_length'].mean():.2f}")
    print(f"Moyenne mots UCE: {df['uce_length'].mean():.2f}")
    
    logger.info("Traitement des UCE terminé.")
    
except Exception as e:
    logger.error(f"Erreur lors du traitement des UCE: {str(e)}")
    raise

INFO:__main__:Début du traitement des UCE...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\web\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
INFO:__main__:Analyse des résultats UCE...
INFO:__main__:Traitement des UCE terminé.



Exemple des 5 premiers résultats UCI → UCE:

Row 0:
UCI: definition opinion publique selon instant pense public vis vis sujet occurrence opinion publique politique pense comment positionne gens quand dis gens grand public peut etre associations chefs entreprise donc definition donne opinion public comment opinion public mesure taux satisfaction regard question posee
UCE: definition opinion publique selon instant pense public vis vis sujet occurrence opinion publique politique pense comment positionne gens quand dis gens grand public peut etre associations chefs entreprise donc definition donne opinion public comment opinion public mesure taux satisfaction regard question posee

Row 1:
UCI: dirais influence enfin dirais tout simplement reseaux sociaux digital presents communication politique autant pris part importante communication politique parce comme certains disent faut vivre temps neanmoins faut savoir mesurer veux dire campagne politique evidemment reseaux sociaux evidemment num

In [17]:
# --------------------------------------------------------------
# Construction du tableau de vocabulaire avec analyse statistique
# --------------------------------------------------------------
import pandas as pd
from collections import Counter
from typing import Dict, List, Set

logger.info("Début de la construction du vocabulaire...")

def build_vocabulary_table(df: pd.DataFrame, column: str = 'uce', min_length: int = 3) -> pd.DataFrame:
    """
    Construit un tableau de vocabulaire enrichi à partir des UCE
    
    Args:
        df (pd.DataFrame): DataFrame source
        column (str): Nom de la colonne à analyser
        min_length (int): Longueur minimale des lemmes à conserver
    
    Returns:
        pd.DataFrame: DataFrame enrichi du vocabulaire
    """
    # Définition des catégories grammaticales à conserver
    POS_CATEGORIES = {'NOUN': 'Nom', 'VERB': 'Verbe', 'ADJ': 'Adjectif', 'ADV': 'Adverbe'}
    
    # Structures pour l'analyse
    vocabulary_data: List[Dict] = []
    lemma_counter = Counter()

    # Traitement du corpus
    for text in df[column].dropna():
        doc = nlp(str(text))
        for token in doc:
            if (token.pos_ in POS_CATEGORIES and 
                len(token.lemma_) > min_length):
                
                lemma_counter[token.lemma_] += 1
                vocabulary_data.append({
                    'Lemme': token.lemma_,
                    'Forme_originale': token.text,
                    'Categorie': POS_CATEGORIES[token.pos_],
                    'Longueur': len(token.lemma_)
                })
    
    # Création du DataFrame enrichi
    df_vocab = pd.DataFrame(vocabulary_data)
    
    # Ajout des statistiques
    df_vocab['Frequence'] = df_vocab['Lemme'].map(lemma_counter)
    df_vocab = df_vocab.drop_duplicates(subset=['Lemme'])
    
    # Tri et organisation
    df_vocab = df_vocab.sort_values(['Frequence', 'Longueur'], ascending=[False, True])
    
    # Statistiques globales
    stats = {
        'total_lemmes': len(df_vocab),
        'freq_min': df_vocab['Frequence'].min(),
        'freq_max': df_vocab['Frequence'].max(),
        'freq_moyenne': df_vocab['Frequence'].mean(),
        'distribution_categories': df_vocab['Categorie'].value_counts().to_dict()
    }
    
    logger.info("Statistiques du vocabulaire:")
    for key, value in stats.items():
        logger.info(f"{key}: {value}")

    # Export des résultats
    export_path = './export/vocabulaire_analyse.csv'
    df_vocab.to_csv(export_path, index=True, encoding='utf-8')
    logger.info(f"Vocabulaire exporté vers: {export_path}")
    
    return df_vocab

# Application de la fonction
df_vocabulaire = build_vocabulary_table(df)

# Affichage des premiers résultats
print("\nAperçu du vocabulaire analysé:")
print(df_vocabulaire.head())

INFO:__main__:Début de la construction du vocabulaire...
INFO:__main__:Statistiques du vocabulaire:
INFO:__main__:total_lemmes: 3584
INFO:__main__:freq_min: 1
INFO:__main__:freq_max: 664
INFO:__main__:freq_moyenne: 7.153180803571429
INFO:__main__:distribution_categories: {'Nom': 1412, 'Adjectif': 1017, 'Verbe': 922, 'Adverbe': 233}
INFO:__main__:Vocabulaire exporté vers: ./export/vocabulaire_analyse.csv



Aperçu du vocabulaire analysé:
        Lemme Forme_originale Categorie  Longueur  Frequence
22       donc            donc   Adverbe         4        664
103     faire           faire     Verbe         5        662
63   campagne        campagne       Nom         8        520
19    pouvoir            peut     Verbe         7        441
15       dire             dis     Verbe         4        391


In [18]:
import os
from pathlib import Path

def export_corpus_files(df: pd.DataFrame, export_dir: str = './export') -> None:
    """
    Exporte les fichiers du corpus avec les noms spécifiés
    
    Args:
        df (pd.DataFrame): DataFrame à exporter
        export_dir (str): Répertoire d'export
    """
    # Création du répertoire d'export s'il n'existe pas
    export_path = Path(export_dir)
    export_path.mkdir(parents=True, exist_ok=True)
    
    try:
        # Configuration des exports avec les noms de fichiers exacts
        exports = {
            'corpus_step_1': {
                'filename': 'export_corpus_step_1.csv',
                'columns': ['uci', 'uce']
            },
            'corpus': {
                'filename': 'export_corpus.csv',
                'columns': ['uci']
            }
        }

        # Réalisation des exports
        for export_type, export_info in exports.items():
            export_file = export_path / export_info['filename']
            
            # Vérification des colonnes et export
            columns_to_export = [col for col in export_info['columns'] if col in df.columns]
            if columns_to_export:
                df.to_csv(
                    export_file,
                    columns=columns_to_export,
                    index=True,
                    encoding='utf-8'
                )
                
                logger.info(f"\nExport {export_type} réalisé:")
                logger.info(f"- Fichier: {export_info['filename']}")
                logger.info(f"- Colonnes exportées: {columns_to_export}")
            else:
                logger.warning(f"Colonnes manquantes pour l'export {export_type}")
        
        logger.info("\nExports terminés avec succès.")
        
    except Exception as e:
        logger.error(f"Erreur lors de l'export: {str(e)}")
        raise

# Exécution des exports
export_corpus_files(df)

INFO:__main__:
Export corpus_step_1 réalisé:
INFO:__main__:- Fichier: export_corpus_step_1.csv
INFO:__main__:- Colonnes exportées: ['uci', 'uce']
INFO:__main__:
Export corpus réalisé:
INFO:__main__:- Fichier: export_corpus.csv
INFO:__main__:- Colonnes exportées: ['uci']
INFO:__main__:
Exports terminés avec succès.


# Construction de la matrice TDM

In [20]:
def prepare_dtm_matrix(export_path: str = './export/export_corpus_step_1.csv') -> pd.DataFrame:
    """
    Prépare la matrice document-terme en important le corpus et en ajoutant les IDs
    
    Args:
        export_path (str): Chemin vers le fichier d'export du corpus
    
    Returns:
        pd.DataFrame: DataFrame préparé pour la DTM
    """
    logger.info("Début de la préparation de la matrice document-terme...")
    
    try:
        # Import du corpus avec vérification
        if not os.path.exists(export_path):
            raise FileNotFoundError(f"Fichier non trouvé: {export_path}")
            
        df_build_dtm = pd.read_csv(export_path, index_col=[0])
        
        # Vérification des colonnes nécessaires
        required_columns = ['uci', 'uce']
        missing_columns = [col for col in required_columns if col not in df_build_dtm.columns]
        if missing_columns:
            raise ValueError(f"Colonnes manquantes dans le fichier: {missing_columns}")
            
        # Ajout de l'ID document de manière optimisée
        df_build_dtm.insert(
            loc=1,
            column="id_doc",
            value=range(len(df_build_dtm))
        )

        # Statistiques sur la matrice
        stats = {
            'nombre_documents': len(df_build_dtm),
            'colonnes_presentes': df_build_dtm.columns.tolist()
        }
        
        logger.info("\nStatistiques de la matrice:")
        for key, value in stats.items():
            logger.info(f"- {key}: {value}")
            
        logger.info("\nAperçu de la matrice préparée:")
        logger.info("\n" + str(df_build_dtm.head()))
        
        return df_build_dtm
        
    except Exception as e:
        logger.error(f"Erreur lors de la préparation de la matrice: {str(e)}")
        raise

# Préparation de la matrice
try:
    df_build_dtm = prepare_dtm_matrix()
    logger.info("Matrice document-terme préparée avec succès.")
except Exception as e:
    logger.error(f"Échec de la préparation de la matrice: {str(e)}")
    raise

INFO:__main__:Début de la préparation de la matrice document-terme...
INFO:__main__:
Statistiques de la matrice:
INFO:__main__:- nombre_documents: 196
INFO:__main__:- colonnes_presentes: ['uci', 'id_doc', 'uce']
INFO:__main__:
Aperçu de la matrice préparée:
INFO:__main__:
                                                 uci  id_doc  \
0  definition opinion publique selon instant pens...       0   
1  dirais influence enfin dirais tout simplement ...       1   
2  element rappelle rappelle souvent autant taux ...       2   
3  pense tout heure pense puisse peut passer apre...       3   
4  oui participe webcams rencontres autant redis ...       4   

                                                 uce  
0  definition opinion publique selon instant pens...  
1  dirais influence enfin dirais tout simplement ...  
2  element rappelle rappelle souvent autant taux ...  
3  pense tout heure pense puisse peut passer apre...  
4  oui participe webcams rencontres autant redis ...  
INFO:__main_

In [21]:
def create_term_document_matrix(df: pd.DataFrame, text_column: str = 'uce') -> pd.DataFrame:
    """
    Crée une matrice terme-document binaire (0/1) à partir d'un DataFrame
    
    Args:
        df (pd.DataFrame): DataFrame source
        text_column (str): Nom de la colonne contenant les textes
    
    Returns:
        pd.DataFrame: Matrice terme-document avec valeurs binaires (0: absence, 1: présence)
    """
    logger.info("Début de la création de la matrice terme-document binaire...")
    
    try:
        # Vérification des données d'entrée
        if text_column not in df.columns:
            raise ValueError(f"Colonne {text_column} non trouvée dans le DataFrame")
        
        # Création de la liste des mots uniques
        unique_words = []
        for text in df[text_column]:
            if pd.notna(text) and isinstance(text, str):
                words = text.split()
                for word in words:
                    if word not in unique_words:
                        unique_words.append(word)
        
        if not unique_words:
            raise ValueError("Aucun mot unique trouvé dans les textes")
            
        logger.info(f"Nombre de termes uniques trouvés: {len(unique_words)}")

        # Initialisation de la matrice avec des zéros (type int)
        term_doc_matrix = pd.DataFrame(0, 
                                     index=df.index,
                                     columns=unique_words,
                                     dtype=np.int8)  # Utilisation de int8 pour optimiser la mémoire
        
        # Mise à jour de la matrice avec des 1 pour les présences
        for word in unique_words:
            term_doc_matrix[word] = df[text_column].apply(
                lambda x: 1 if pd.notna(x) and isinstance(x, str) and word in x.split() else 0
            ).astype(np.int8)
        
        # Fusion avec le DataFrame original
        result_df = pd.concat([df, term_doc_matrix], axis=1)
        
        # Statistiques sur la matrice binaire
        nb_terms = len(unique_words)
        nb_docs = len(df)
        presence_count = (term_doc_matrix == 1).sum().sum()
        
        logger.info("\nStatistiques de la matrice binaire:")
        logger.info(f"- Dimensions: {nb_docs} documents × {nb_terms} termes")
        logger.info(f"- Nombre total de présences (1): {presence_count}")
        logger.info(f"- Densité: {(presence_count / (nb_docs * nb_terms)):.2%}")
        logger.info("- Top 10 termes les plus présents:")
        top_terms = term_doc_matrix.sum().nlargest(10)
        for term, count in top_terms.items():
            logger.info(f"  * {term}: présent dans {int(count)} documents")
        
        return result_df
        
    except Exception as e:
        logger.error(f"Erreur lors de la création de la matrice: {str(e)}")
        raise

# Création de la matrice
try:
    df_build_dtm = create_term_document_matrix(df_build_dtm)
    logger.info("Matrice terme-document binaire créée avec succès.")
except Exception as e:
    logger.error(f"Échec de la création de la matrice: {str(e)}")
    raise

INFO:__main__:Début de la création de la matrice terme-document binaire...
INFO:__main__:Nombre de termes uniques trouvés: 5214
INFO:__main__:
Statistiques de la matrice binaire:
INFO:__main__:- Dimensions: 196 documents × 5214 termes
INFO:__main__:- Nombre total de présences (1): 21008
INFO:__main__:- Densité: 2.06%
INFO:__main__:- Top 10 termes les plus présents:
INFO:__main__:  * donc: présent dans 159 documents
INFO:__main__:  * fait: présent dans 150 documents
INFO:__main__:  * parce: présent dans 135 documents
INFO:__main__:  * tout: présent dans 134 documents
INFO:__main__:  * dire: présent dans 125 documents
INFO:__main__:  * campagne: présent dans 124 documents
INFO:__main__:  * quand: présent dans 113 documents
INFO:__main__:  * faire: présent dans 112 documents
INFO:__main__:  * peu: présent dans 108 documents
INFO:__main__:  * meme: présent dans 104 documents
INFO:__main__:Matrice terme-document binaire créée avec succès.


In [22]:
def display_tdm_structure(df: pd.DataFrame, original_columns: list) -> None:
    """
    Affiche la structure détaillée de la matrice TDM
    
    Args:
        df (pd.DataFrame): DataFrame contenant la matrice TDM
        original_columns (list): Liste des colonnes d'origine avant ajout de la matrice
    """
    # Séparation des colonnes originales et de la matrice TDM
    tdm_columns = [col for col in df.columns if col not in original_columns]
    
    # Création du DataFrame de structure
    structure_info = pd.DataFrame({
        'Type': [
            'Colonnes originales',
            'Termes de la matrice TDM',
            'Total'
        ],
        'Nombre': [
            len(original_columns),
            len(tdm_columns),
            len(df.columns)
        ]
    })

    # Calcul des statistiques de la matrice TDM
    tdm_part = df[tdm_columns]
    tdm_stats = {
        'Nombre de documents': len(df),
        'Nombre de termes uniques': len(tdm_columns),
        'Nombre total de présences (1)': (tdm_part == 1).sum().sum(),
        'Densité de la matrice': f"{((tdm_part == 1).sum().sum() / (len(df) * len(tdm_columns))):.2%}",
        'Type de données': tdm_part.dtypes.value_counts().to_dict()
    }
    
    # Affichage des résultats
    print("\n=== Structure de la matrice TDM ===")
    print(structure_info.to_string(index=False))
    
    print("\n=== Statistiques détaillées ===")
    for key, value in tdm_stats.items():
        print(f"{key}: {value}")
    
    print("\n=== Aperçu de la matrice (5 premiers documents, 5 premiers termes) ===")
    sample_cols = original_columns + tdm_columns[:5]
    print(df[sample_cols].head().to_string())
    
    print("\n=== Top 10 termes les plus fréquents ===")
    top_terms = tdm_part.sum().nlargest(10)
    for term, count in top_terms.items():
        print(f"- {term}: présent dans {int(count)} documents")

# Après la création de la matrice TDM
original_columns = ['id_doc', 'uci', 'uce']  # Ajustez selon vos colonnes d'origine
display_tdm_structure(df_build_dtm, original_columns)


=== Structure de la matrice TDM ===
                    Type  Nombre
     Colonnes originales       3
Termes de la matrice TDM    5214
                   Total    5217

=== Statistiques détaillées ===
Nombre de documents: 196
Nombre de termes uniques: 5214
Nombre total de présences (1): 21008
Densité de la matrice: 2.06%
Type de données: {dtype('int8'): 5214}

=== Aperçu de la matrice (5 premiers documents, 5 premiers termes) ===
   id_doc                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [23]:
def prepare_reinert_matrix(df: pd.DataFrame, export_path: str = './export/reinert_dataframe.csv') -> pd.DataFrame:
    """
    Prépare et exporte la matrice pour l'analyse Reinert
    
    Args:
        df (pd.DataFrame): DataFrame source (matrice TDM)
        export_path (str): Chemin pour l'export du fichier
        
    Returns:
        pd.DataFrame: Matrice préparée pour l'analyse Reinert
    """
    logger.info("Préparation de la matrice pour l'analyse Reinert...")
    
    try:
        # Vérification des colonnes requises
        required_columns = ['id_doc', 'uci', 'uce']
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Colonnes manquantes: {missing_columns}")
        
        # Création de la matrice Reinert
        df_reinert = df.drop(columns=['uce'])
        
        # Vérification de la structure avant export
        matrix_stats = {
            'dimensions': df_reinert.shape,
            'colonnes': len(df_reinert.columns) - 2,  # -2 pour id_doc et uci
            'documents': len(df_reinert)
        }

        # Export de la matrice
        export_dir = os.path.dirname(export_path)
        if not os.path.exists(export_dir):
            os.makedirs(export_dir)
            
        df_reinert.to_csv(export_path, index=False)
        logger.info(f"Matrice exportée vers: {export_path}")
        
        # Chargement et préparation finale
        df_reinert_afc = pd.read_csv(export_path, header=0, index_col=0)
        df_reinert_afc = df_reinert_afc.set_index('id_doc')
        
        # Affichage des informations
        logger.info("\nStructure de la matrice Reinert:")
        logger.info(f"- Dimensions: {matrix_stats['dimensions']}")
        logger.info(f"- Nombre de termes: {matrix_stats['colonnes']}")
        logger.info(f"- Nombre de documents: {matrix_stats['documents']}")
        
        # Vérification de la cohérence
        logger.info("\nVérification de la cohérence:")
        logger.info(f"- Types de données: {df_reinert_afc.dtypes.value_counts().to_dict()}")
        logger.info(f"- Valeurs manquantes: {df_reinert_afc.isnull().sum().sum()}")
        
        return df_reinert_afc
        
    except Exception as e:
        logger.error(f"Erreur lors de la préparation de la matrice Reinert: {str(e)}")
        raise

# Exécution de la préparation
try:
    df_reinert_afc = prepare_reinert_matrix(df_build_dtm)
    
    # Affichage de l'aperçu
    print("\nAperçu de la matrice finale:")
    print(df_reinert_afc.head())
    
except Exception as e:
    logger.error(f"Échec de la préparation: {str(e)}")
    raise

INFO:__main__:Préparation de la matrice pour l'analyse Reinert...
INFO:__main__:Matrice exportée vers: ./export/reinert_dataframe.csv
INFO:__main__:
Structure de la matrice Reinert:
INFO:__main__:- Dimensions: (196, 5216)
INFO:__main__:- Nombre de termes: 5214
INFO:__main__:- Nombre de documents: 196
INFO:__main__:
Vérification de la cohérence:
INFO:__main__:- Types de données: {dtype('int64'): 5214}
INFO:__main__:- Valeurs manquantes: 0



Aperçu de la matrice finale:
        definition  opinion  publique  selon  instant  pense  public  vis  \
id_doc                                                                      
0                1        1         1      1        1      1       1    1   
1                0        0         0      0        0      0       0    0   
2                0        1         1      0        0      0       0    0   
3                0        0         0      0        0      1       0    0   
4                0        0         0      0        0      0       0    0   

        sujet  occurrence  ...  directions  ressemblent  mastodontes  usages  \
id_doc                     ...                                                 
0           1           1  ...           0            0            0       0   
1           0           0  ...           0            0            0       0   
2           0           0  ...           0            0            0       0   
3           0           0  ...